# Generate h2 and h1 threshold based on fscore of validation set

First, we generate default threshold for 2 level classifier based on f-score, then adjust those value by hand.
In this notebook, we generate default thresholds.

## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# Use this for get catkeys.
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "")

### Virtual Category case

In [ ]:
CATSTORE_PATH = ""

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)
catkeys = vc.keys()

### 178 category case

In [ ]:
from category import load_category_dict

In [ ]:
catdict = load_category_dict(CATEGORY_DICT_CSV)

### First threshold pick (best f1 score)

In [ ]:
DIST_THRESHOLD=1.5

#created by classifier_similarity.ipynb
VALID_DF_PATH=""

VALID_DISTANCE_DICT_PATH=""

In [ ]:
scoredf = pd.read_pickle(VALID_DF_PATH)

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
from functools import reduce

In [ ]:
def calc_raw_scores(targetkey, thrshold):
    similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)
    
    allfalse = np.zeros(len(scoredf), dtype=bool)
    tp_indices = reduce((lambda x, y: x | y), [allfalse] + [scoredf['label'] == key for key in similarkeyset])
    trues = scoredf[tp_indices]
    falses = scoredf[~tp_indices]
    TP = len(trues[trues[targetkey] > thrshold])
    TN = len(falses[falses[targetkey] <= thrshold])
    FP = len(falses[falses[targetkey] > thrshold])
    FN = len(trues[trues[targetkey] <= thrshold])
    # TP = scoredf[tp_indices]
    # TN = scoredf[~tp_indices]
    #FP = TN[TN[targetkey] > thrshold]
    #FN = TP[TP[targetkey] <= thrshold]
    return {"TP": TP, "TN":TN, "FP":FP, "FN":FN}

In [ ]:
def rawscores2fscores(raw_scores, thrds, BETA):
    EPS = 0.00001
    scoretups = [(dic["TP"]/(dic["TP"]+dic["FP"]+EPS), dic["TP"]/(dic["TP"]+dic["FN"]+EPS), (1+BETA**2)*dic["TP"]/((1+BETA**2)*dic["TP"]+dic["FP"]+(BETA**2)*dic["FN"]+EPS)) for dic in raw_scores]
    scores = pd.DataFrame({"precision":[tup[0] for tup in scoretups], "recall":[tup[1] for tup in scoretups], "fscore":[tup[2] for tup in scoretups]})
    scores.index = thrds
    return scores

In [ ]:
def thrsholds2f1df(targetkey, thrds, BETA):
    raw_scores = [calc_raw_scores(targetkey, thrd) for thrd in thrds]
    return rawscores2fscores(raw_scores, thrds, BETA)

In [ ]:
def find_one(thrds2fscoredfproc, minthrd, maxthrd):
    middle = (minthrd+maxthrd)/2
    halfw = (middle-minthrd)/2
    score = thrds2fscoredfproc([minthrd, middle, maxthrd])
    chosen =  score["fscore"].idxmax()
    return (chosen, [chosen-halfw, min(0.9999, chosen+halfw)])


In [ ]:
def find_best_fscore_threshold(thrds2fscoredfproc, minthrd, maxthrd, depth=7):
    for _ in range(depth):
        val, (minthrd, maxthrd) = find_one(thrds2fscoredfproc, minthrd, maxthrd)
    return val


In [ ]:
def best_fscores_threshold(targetkey, BETA=1.0):
    proc = lambda thrds: thrsholds2f1df(targetkey, thrds, BETA)
    return find_best_fscore_threshold(proc, 0.9, 1.0)
    

### Manual beta selection

- best threshold for 147 is about 0.998 and beta is 0.05
- best threshold for 129 is about 0.924 and beta is 0.15
- best threshold for 160 is about 0l.965 and beta is 0.125

I choose Beta=0.125

In [ ]:
best_fscores_threshold('160', 0.15)

In [ ]:
len(cattarget)

About 2min for 18 categories

In [ ]:
%%time
[(str(cat), best_fscores_threshold(str(cat), 0.125), 0.5, label) for cat,label in [(cat, vc.name(cat)) for cat in cattarget]] 

In [ ]:
from operator import itemgetter

catproc = [(
)]

catproc = sorted(catproc, key=itemgetter(0))

In [ ]:
%%time
catproc_notsorted = [(str(cat), best_fscores_threshold(str(cat), 0.125), 0.5, label) for cat,label in [(cat, vc.name(cat)) for cat in catkeys]] 

In [ ]:
### 178 category case
%%time
catproc_notsorted = [(str(cat), best_f1_threshold(str(cat)), 0.5, label) for cat,label in catdict.items()] 

In [ ]:
from operator import itemgetter

catproc = sorted(catproc_notsorted, key=itemgetter(0))

In [ ]:
catproc

### Update some classifier threshold by hand 

In [ ]:
catproc[107], catproc[137]

In [ ]:
def replace_one(org, thrd):
    return  (org[0], thrd, org[2], org[3])

In [ ]:
catproc[107] = replace_one(catproc[107], 0.992)

In [ ]:
catproc[137] = replace_one(catproc[137], 0.87)

In [ ]:
catproc[107], catproc[137]

### Choose 2nd level

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
h2binder = ModelBinder_create()
h1binder = ModelBinder.dup_from(h2binder)

In [ ]:
# del sys.modules['one_vs_all']

In [ ]:
from one_vs_all import TwoLevelClassifier

In [ ]:
two = TwoLevelClassifier(catproc, h2binder, h1binder)

In [ ]:
two.load_all()

In [ ]:
CHECK_DATA_PATHS = list(glob.glob(os.path.normpath("{}/**/*.jpg").format(VALID_DATA_DIR)))

In [ ]:
%%time
# to store second score df
_ = two.predict_files_df(CHECK_DATA_PATHS)

# Save or load 2nd score df

In [ ]:
VALID_SECSCORE_PATH=""

### save to file

In [ ]:
df = two._df
df['filepaths'] = CHECK_DATA_PATHS

In [ ]:
df.to_pickle(VALID_SECSCORE_PATH)

### load from file

In [ ]:
df = pd.read_pickle(VALID_SECSCORE_PATH)

# Calc second level f1 score

In [ ]:
vkeys = [key for key in vc.keys() if len(key.split("_")) >=2]

In [ ]:
vdict = {key:vkey for vkey in vkeys for key in vkey.split("_")}

In [ ]:
def key_to_vkey(key):
    if key in vdict:
        return vdict[key]
    return key

In [ ]:
labels = [key_to_vkey(os.path.basename(os.path.dirname(path))) for path in df['filepaths'].values]

In [ ]:
df['label'] = labels

In [ ]:
def calc_raw_scores2nd(df, targetkey, thrshold):
    
    trues = df[df['label'] == targetkey]
    falses = df[df['label'] != targetkey]
    TP = len(trues[trues[targetkey] > thrshold])
    TN = len(falses[falses[targetkey] <= thrshold])
    FP = len(falses[falses[targetkey] > thrshold])
    FN = len(trues[trues[targetkey] <= thrshold])
    return {"TP": TP, "TN":TN, "FP":FP, "FN":FN}

In [ ]:
def thrsholds2fscore2nd(df, targetkey, thrds, BETA):
    raw_scores = [calc_raw_scores2nd(df, targetkey, thrd) for thrd in thrds]
    return rawscores2fscores(raw_scores, thrds, BETA)


In [ ]:
def best_fscore_threshold2nd(df, targetkey, BETA=1.0):
    proc = lambda thrds: thrsholds2fscore2nd(df, targetkey, thrds, BETA)
    return find_best_fscore_threshold(proc, 0.9, 1.0)


### Find good beta. Goal thrshold is defined by hand.

I choose following value by hand.

Conclusion: Difficult to choose by automatically.

In [ ]:
best_fscore_threshold2nd(df, '93', BETA=0.8)

In [ ]:
best_fscore_threshold2nd(df, '45', BETA=0.8), best_fscore_threshold2nd(df, '45', BETA=0.28), best_fscore_threshold2nd(df, '45', BETA=0.29) 

In [ ]:
best_fscore_threshold2nd(df, '65_81', BETA=0.1), best_fscore_threshold2nd(df, '65_81', BETA=0.125), best_fscore_threshold2nd(df, '65_81', BETA=0.15)

In [ ]:
best_fscore_threshold2nd(df, '4', BETA=1)

In [ ]:
best_fscore_threshold2nd(df, '57_64', BETA=0), best_fscore_threshold2nd(df, '57_64', BETA=1), best_fscore_threshold2nd(df, '57_64', BETA=10)

In [ ]:
best_fscore_threshold2nd(df, '102', BETA=0.15)

In [ ]:
best_fscore_threshold2nd(df, '20', BETA=0.15)

In [ ]:
%%time
catproc2nd = [(tup[0], tup[1], best_fscore_threshold2nd(df, tup[0], BETA=0.15), tup[3]) for tup in catproc]

In [ ]:
catproc2nd

In [ ]:
calc_raw_scores2nd(df, '93', 0.8)